In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf 

In [ ]:
def download(ticker, start, end):
    return yf.download(ticker, start=start, end=end)

In [ ]:
data = download(ticker=TICKER, start=START, end=END)

In [ ]:

TICKER = "IBM"
START = "2020-01-01"
END = "2025-01-01"
PERIOD = 20
COEF = 2
"""BBAND visualization"""
df = data.copy()
df["tp"] = (df["Close"] + df["Low"]+df["High"])/3
df["STD"] = df["tp"].rolling(PERIOD).std()
df["MA"] = df["tp"].rolling(PERIOD).mean()
df["Upper"] = df["MA"] + COEF*df["STD"]
df["Lower"] = df["MA"] - COEF*df["STD"]
df.dropna(inplace=True)

def plot(data):
    plt.plot(data["Close"])
    plt.plot(data["Upper"], color="orange")
    plt.plot(data["Lower"], color="orange")
    plt.fill_between(data.index, data["Upper"],
                     data["Lower"], facecolor="orange", alpha=0.1)
    
    plt.title("BBANDs")
    plt.show()
    
plot(data=df)

In [ ]:
"""Strategy"""